In [1]:
#from csv import reader
import pandas as pd

In [2]:
# obtain the dataset contain unix last command results
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
last = pd.read_csv('last.csv',header=None)
last['user'] = last.apply(lambda x: x.str.split()[0][0], axis =1)
last['month'] = last.apply(lambda x: x.str.split()[0][4], axis =1)
last1 = last.loc[(last.user != 'wtmp') & (last.month.isin(months)),['user','month']]
last1['count']=0
last2=last1.groupby(by=['user','month'])['count'].count()
last3=last2.reset_index()
last3

,user,month,count
0,afaqeeh,Apr,1
1,afaqeeh,Jul,1
2,akavaler,Jun,1
3,aploszaj,Apr,1
4,aploszaj,Feb,1
...,...,...,...
73,ssikdar,Mar,3
74,ssikdar,May,7
75,vlulla,Apr,95
76,vlulla,Feb,116


In [3]:
# obtain data containing psql names and oids
dboid = pd.read_csv('db_names.csv')
dboid

,oid,datname
0,103025336,afaqeeh_db
1,402743390,ajdicks_db
2,470739119,akavaler_db
3,234055918,amanjuna_db
4,470452526,aploszaj_db
...,...,...
62,234056361,yoonjis_db
63,402719224,youms_db
64,402924572,ysevryug_db
65,110327009,zdunivin_db


In [4]:
# obtain data containing the sizes of the folders (oid)
sizes = pd.read_csv('db_sizes.csv',delimiter='\t',header=None).rename(columns={0:'size',1:'oid'})
sizes = sizes.loc[sizes.oid != 'pgsql_tmp',:]
sizes.oid = sizes.oid.astype('int64')
print(sizes.loc[sizes['oid']==16386,:])
sizes

    size    oid
39  3.3T  16386


,size,oid
0,8.6M,1
1,89G,100633275
2,431G,103025336
3,8.6M,103648214
4,44M,103649330
...,...,...
62,81M,700660004
63,8.6M,700668229
64,8.6M,700676986
65,8.6M,734826481


In [5]:
# join size and oid data on oid
data = pd.merge(sizes, dboid, on='oid')
data

,size,oid,datname
0,8.6M,1,template1
1,89G,100633275,gongk_db
2,431G,103025336,afaqeeh_db
3,8.6M,103648214,smilojev_db
4,44M,103649330,chni_db
...,...,...,...
62,81M,700660004,vlulla_db
63,8.6M,700668229,ssikdar_db
64,8.6M,700676986,eehkim_db
65,8.6M,734826481,jwd_db


In [6]:
# create user column from datname column on joined data
data2 = data.copy()
data2.loc[:,'user'] = data2.datname.apply(lambda x: x[:-3])
data2

,size,oid,datname,user
0,8.6M,1,template1,templa
1,89G,100633275,gongk_db,gongk
2,431G,103025336,afaqeeh_db,afaqeeh
3,8.6M,103648214,smilojev_db,smilojev
4,44M,103649330,chni_db,chni
...,...,...,...,...
62,81M,700660004,vlulla_db,vlulla
63,8.6M,700668229,ssikdar_db,ssikdar
64,8.6M,700676986,eehkim_db,eehkim
65,8.6M,734826481,jwd_db,jwd


In [7]:
# join last and previously joined data on user
data3=pd.merge(last3, data2, on='user',how='right')
data3

,user,month,count,size,oid,datname
0,templa,NaN,NaN,8.6M,1,template1
1,gongk,NaN,NaN,89G,100633275,gongk_db
2,afaqeeh,Apr,1.0,431G,103025336,afaqeeh_db
3,afaqeeh,Jul,1.0,431G,103025336,afaqeeh_db
4,smilojev,Jun,6.0,8.6M,103648214,smilojev_db
...,...,...,...,...,...,...
106,ssikdar,May,7.0,8.6M,700668229,ssikdar_db
107,eehkim,May,6.0,8.6M,700676986,eehkim_db
108,jwd,Jul,1.0,8.6M,734826481,jwd_db
109,shaw10,Aug,18.0,8.6M,734826483,shaw10_db


In [8]:
# create sizeGB column from size column that is sort friendly
data4=data3.copy()
data4.sizeU=0
data4.loc[:,'sizeGB']=data4['size'].apply(lambda x: x[:-1] if x[-1]=='G' else float(str(x)[:-1])/1024)
data4['sizeGB']=data4['sizeGB'].astype('float64')
data4.sort_values('sizeGB',ascending=False,inplace=True)
data4.reset_index()

,index,user,month,count,size,oid,datname,sizeGB
0,2,afaqeeh,Apr,1.0,431G,103025336,afaqeeh_db,431.000000
1,3,afaqeeh,Jul,1.0,431G,103025336,afaqeeh_db,431.000000
2,65,maahutch,NaN,NaN,421G,234056457,maahutch_db,421.000000
3,33,rmurcian,Mar,10.0,107G,110426031,rmurcian_db,107.000000
4,32,rmurcian,Jun,4.0,107G,110426031,rmurcian_db,107.000000
...,...,...,...,...,...,...,...,...
106,4,smilojev,Jun,6.0,8.6M,103648214,smilojev_db,0.008398
107,110,shaw10,Jul,9.0,8.6M,734826483,shaw10_db,0.008398
108,61,postg,NaN,NaN,8.3M,13881,postgres,0.008105
109,60,templa,NaN,NaN,7.7M,13880,template0,0.007520


In [115]:
data4.to_csv('last_M_sizes.csv',index=False)